# Homework 3 

# Задача №1 - Лес или пустыня?

Часто при анализе изображений местности необходимо понять ее характер. В частности, если определить, что на изображении преобладет вода, то имеет смысл искать корабли на таком изображении. Если на картинке густой лес, то, возможно, это не лучшая зона для посадки дрона или беспилотника.

Ваша задача - написать программу, которая будет отличать лес от пустыни. В приложении можно найти реальные спутниковые снимки лесов и пустынь.

Примеры изображений:
<table><tr>
    <td> <img src="https://i.ibb.co/nmHHctW/test_image_00.jpg" alt="Drawing" style="width: 200px;"/> </td>
    <td> <img src="https://i.ibb.co/dM77C4b/test_image_06.jpg" alt="Drawing" style="width: 200px;"/> </td>
</tr></table>

In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt

%matplotlib inline

def process2gray(image):  
    '''
    Переводит изображение в серый цвет

    :param image: изображение леса или пустыни  
    :return: серое изображение
    '''
    h, w, _ = image.shape
    image = cv2.resize(image.copy(), (w // 2, h // 2))
    image = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)

    gray = cv2.cvtColor(image.copy(), cv2.COLOR_RGB2GRAY)
    return gray

def image_class(hist):    
    '''
    Определяет лес или пустыня на фотоснимке

    :param hist: гистограмма изображения леса или пустыни
    :return: "desert", если на изображении пустыня, иначе - "forest"
    '''
    desert_threshhold = 60000 #Подбирался вручную (можно и точнее)
    #Берем интервал [0:100]. леса как правило более темные чем пустыни
    if(np.sum(hist[0:100]) < desert_threshhold):
        return "desert"
    else:
        return "forest"
    
    
#Считываем изображения
images = list()

for i in range(16):
    if(i < 10):
        image = cv2.imread(f'desert_forest/test_image_0{i}.jpg')        
    else:
        image = cv2.imread(f'desert_forest/test_image_{i}.jpg')              
    images.append(image)

#Список серых изображений
grays = list()
for i in range(16):
    grays.append(process2gray(images[i]))

#Список гистограмм
hists = list()
for i in range(16):
    hist = cv2.calcHist([grays[i].ravel()], [0], None, [256], [0, 256])
    hists.append(hist)

#Классифицируем изображения
photo_classes = list()

for i in range(16):
    photo_class = image_class(hists[i])
    if (photo_class == 'desert'):
        photo_classes.append('desert')
    else:
        photo_classes.append('forest')

plt.figure(figsize=(16, 8), constrained_layout=True)

plt.subplot(1, 2, 1)
for i, img in enumerate(images):
    plt.subplot(2, len(images) // 2, i + 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))   
    if(photo_classes[i] == 'forest'):
        plt.title('forest', fontsize=14)
        plt.axis('off')
    else:
        plt.title('desert', fontsize=14)
        plt.axis('off')

plt.axis('off')
plt.show()

# Задача №2 - Реализовать Image-blending на основе сшивки по градиентам

Задача - взять фото двух лиц : ваше и друга, с помощью метода Poisson image editing совместить глаза, нос и рот с первого изображения со вторым. Суть в том, что при использовании такого метода границы совмещенного изображения не видны.

Статья, где описан метод  

Patrick Pérez, Michel Gangnet, and Andrew Blake. 2003. Poisson image editing. ACM Trans. Graph. 22, 3 (July 2003), 313–318. https://doi.org/10.1145/882262.882269

Чуть более понятный гайд о том как написать Пуассоновское сшивание: https://cs.brown.edu/courses/cs129/results/proj2/taox/

Пример такого совмещения:

<img src="./blending/blending.png" alt="Drawing" style="width: 700px;"/>


In [2]:
# Ваш код

# Задача №3 - Найди клетки

Даны снимки раковых клеток. Существует задача - определить стадию рака клетки по такому изображению. Для того, чтобы подойти к решению классификации рака клетки, необходимо сначала подготовить данные.

Исходные изображения в реальных задачах могут быть очень большого размера (более 20000 px). Однако из визуального анализа можно заметить, что большая часть этих снимков пустая и не несет в себе полезную информацию.

Ваша задача выделить небольшие ячейки изображений из исходного так, чтобы на ячейках было только изображение клетки.

Пример исходного изображения и нарезанных ячеек клетки.
<img src="./cell_example.png" alt="Drawing" style="width: 500px;"/>

В качестве аргументов у функции будут значения:
1. исходное изображние;
2. размер ячейки;
3. количество ячеек.

__Доп вопрос__ - как можно выяснить какие нужны значения аргументов, чтобы они подходили для большинства исходных снимков?

In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt

%matplotlib inline

class Cell:
    '''
    Класс, описывающий ячейку изображения
    parametrs:
        x: координата х левого верхнего угла ячейки
        y: координата y левого верхнего угла ячейки
        cell_image: изображение ячейки в виде матрицы
        саncer_amount: какая часть ячейки заполнена раковыми образованиями (0 <= саncer_amount <= 1)                
    '''
    def __init__(self, x, y, cell_image, cancer_amount = 0):
        self.x = x
        self.y = y
        self.cell_image = cell_image
        self.cancer_amount = cancer_amount


def combine_images_on_background(images):
    '''
    Размещает ячейки на белом фоне

    :param images: список Cell    
    :return: изображение, составленное из ячеек, размещенных на белом фоне
    '''
    images_count = len(images)
    cols = 4
    rows = (images_count // cols) + 1

    h, w, _ = (images[0].cell_image).shape

    background = np.ones(( rows * h, cols * w, 3), dtype = np.uint8) * 255

    for i, img in enumerate(images):        
        row = i // cols
        col = i % cols

        x = col * (w)
        y = row * (h)
        
        background[y:y + h, x:x + w] = img.cell_image

    return background
    

def sliding_windows(image, win_size, win_step):
    '''
    Выделяет скользящие окна (Cell) из матрицы

    :param image: входное изображение
    :param win_size: размер окна
    :param win_step: шаг, с которым будет перемещаться окно по оси x
    :return: список Cell из подматриц исходной матрицы, которые получились путем прохода окна
    '''
    windows = list()
    h, w, _ = image.shape

    for y in range(0, h - win_size + 1, win_size):
        for x in range(0, w - win_size + 1, win_step):
            win = image[y: y + win_size, x: x + win_size]
            c = Cell(x, y, win)
            windows.append(c)

    return windows

def is_correct_cell(cell):
    '''
    Определяет стоит ли заносить данную ячейку в список тех, что нужно оставить

    :param сell: элемент списка скользящих окон
    :return: True если threshold часть ячейки заполнена раковыми образованиями, иначе - false & ячейку
    '''

    h, w, _ = cell.cell_image.shape

    threshold = 0.9 #если такая часть ячейки заполнена раковой опухолью, то запоминаем ее

    total_pixels = h * w

    non_white_sum = np.sum(cell.cell_image < 255) #Считаем все не белые пиксели, если их больше чем threshold*total_pixels, то True

    cancer_amount = non_white_sum / total_pixels

    cell.cancer_amount = cancer_amount

    return cell.cancer_amount > threshold, cell

def get_cancer_cells(image, cell_size, cell_number):
    '''
    Определяет ячейки в которых содержится больше всего информации о раковых образованиях

    :param image: фото раковых клеток
    :param cell_size: размер ячейки
    :param cell_number: количество ячеек
    :return: список из Сell, которые содержат больше всего информации о раковых образованиях
    '''
    suitable_cells = list()    

    n = 10 #нужно для определеняи шага с которым будет шагать окно в функции sliding_windows

    cell_step = cell_size // n # вместо 10 можно вписать другое разумное число
    cells_list = sliding_windows(image, cell_size, cell_step)

    i = 0
    while i != len(cells_list):  
        c = cells_list[i]
        is_correct, cell = is_correct_cell(c)      
        if(is_correct):
            suitable_cells.append(cell)                   
            i += n #Перешагиваем через n ячеек (зачем их смотреть, если они налазят на подошедшеую нам ячейку)
        else:
            i += 1

    #Сортируем по количеству раковых образований в ячейке, оставляем те, которые содержат больше всего информации
    suitable_cells.sort(key=lambda cell: cell.cancer_amount, reverse=True)
    del suitable_cells[cell_number:]

    return suitable_cells


#Исходное изображение:
image = cv2.imread('cells/train3_1.jpeg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image, cmap = 'gray')
plt.axis('off')
plt.show()

#Выделяем на исходном изображении ячейки
cell_size = 150 #Размер ячейки
cell_number = 50 #Количество ячеек
cells = get_cancer_cells(image, cell_size, cell_number)

for cell in cells:
    cv2.rectangle(image, (cell.x, cell.y), (cell.x + cell_size, cell.y + cell_size), (0, 0, 0), 6)

plt.imshow(image, cmap = 'gray')
plt.axis('off')
plt.show()

#Выводим полученные ячейки на белом фоне
combined_cells = combine_images_on_background(cells)

plt.imshow(combined_cells, cmap = 'gray')
plt.axis('off')
plt.show()

